In [ ]:
# Clone repository from GitHub
!git clone https://github.com/hyecheol123/CycleGAN-Music-Style-Transfer.git
# !git pull

Cloning into 'CycleGAN-Music-Style-Transfer'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 336 (delta 25), reused 31 (delta 11), pack-reused 284
Receiving objects: 100% (336/336), 1.37 MiB | 3.06 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [ ]:
# Change working directory
import os
os.chdir('CycleGAN-Music-Style-Transfer/')

In [ ]:
# Installing dependencies
!pip install folium==0.2.1
!pip install Pygments==2.2.0
!pip install -r requirements.txt

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=99df0c4d8df699aae86c599f8112bc5bddbf0f04b4c0e614ab17b91d35d08112
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 849kB 9.3MB/s 
  Found existing installation: Pygments 2.1.3
    Uninstalling Pygments-2.1.3:
      Successfully uninstalled Pygments-2.1.3


     |████████████████████████████████| 5.6MB 13.0MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for metrics: filename=metrics-0.3.3-py2.py3-none-any.whl size=17815 sha256=e957e98231984b9fcc4c88e449f64d6aee49bf9cc8ba553f9197d567f012983b
  Stored in directory: /root/.cache/pip/wheels/38/e8/55/c4c648451a33a60920318eba4c5735088dbf78dc20d4277db0
  Created wheel for pypianoroll: filename=pypianoroll-0.5.3-cp36-none-any.whl size=23827 sha256=75800b34bfaccc02adfa84ad10d4b342272037f3d4c432c2421d07e219f6b4db
  Stored in directory: /root/.cache/pip/wheels/29/c8/c0/7b240ab723c2f96b03391796bdf278de513eabf1dfb989c07f
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591953 sha256=7c7f4af4d922fe3ba20c03630fe556c8b0fc744e7468de670b1cb11f97f78585
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
  Created wheel for pathspec: filename=pathspec-0.5.5-cp36-none-any.whl size=24229 sha256

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.client import device_lib

print(tf.__version__)
print("\n")

tf.config.experimental.list_physical_devices('GPU')

TensorFlow 1.x selected.
1.15.2




[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Mounting Google Drive to current CoLab Notebook
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Listing available datasets
ls /content/drive/My\ Drive/453/dataset2

classical/  jazz/  pop/  rock/


## Data Pre-process
Only run this part of code when you need to pre-process the data  
It takes long time.

In [ ]:
import numpy as np
import glob
import datetime
import math
import random
import os
import shutil
import matplotlib.pyplot as plt
import pretty_midi
from pypianoroll import Multitrack, Track
import librosa.display
from utils import *

# Import for part 2
from __future__ import print_function
import json
import errno

ROOT_PATH = '/content/drive/My Drive/453/dataset' # root path of the dataset
DATASET_NAMES = ["classical", "jazz", "pop", "rock"] # Where the list of dataset name to be stored
TEMP_SPLITSET_NAMES = ['test_temp', 'train_temp'] # For temporal testset and trainset names
                                                  # TEMP_SPLITSET_NAMES[0] for testset
                                                  # TEMP_SPLITSET_NAMES[1] for trainset
# all midi files must be stored in subfolder named "<DATASET_NAME>_midi" inside the folder named "<DATASET_NAME>"
test_ratio = 0.1
LAST_BAR_MODE = 'remove'

def get_bar_piano_roll(piano_roll):
    if int(piano_roll.shape[0] % 64) is not 0:
        if LAST_BAR_MODE == 'fill':
            piano_roll = np.concatenate((piano_roll, np.zeros((64 - piano_roll.shape[0] % 64, 128))), axis=0)
        elif LAST_BAR_MODE == 'remove':
            piano_roll = np.delete(piano_roll,  np.s_[-int(piano_roll.shape[0] % 64):], axis=0)
    piano_roll = piano_roll.reshape(-1, 64, 128)
    return piano_roll


def to_binary(bars, threshold=0.0):
    """Turn velocity value into boolean"""
    track_is_max = tf.equal(bars, tf.reduce_max(bars, axis=-1, keep_dims=True))
    track_pass_threshold = (bars > threshold)
    out_track = tf.logical_and(track_is_max, track_pass_threshold)
    return out_track

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1916928/45929032 bytes (4.2%)5718016/45929032 bytes (12.4%)9240576/45929032 bytes (20.1%)12984320/45929032 bytes (28.3%)16826368/45929032 bytes (36.6%)20414464/45929032 bytes (44.4%)24199168/45929032 bytes (52.7%)27901952/45929032 bytes (60.8%)31653888/45929032 bytes (68.9%)35487744/45929032 bytes (77.3%)38846464/45929032 bytes (84.6%)42590208/45929032 bytes (92.7%)

In [ ]:
"""1. divide the original set into train and test sets"""
for dataset in DATASET_NAMES:
  # generating paths
  dataset_path = os.path.join(ROOT_PATH, dataset)
  originals_path = os.path.join(dataset_path, (dataset + "_midi"))
  testset_path = os.path.join(dataset_path, TEMP_SPLITSET_NAMES[0], "origin_midi")
  trainset_path = os.path.join(dataset_path, TEMP_SPLITSET_NAMES[1], "origin_midi")
  # Make Testset and Traingset directory if needed
  if not os.path.exists(testset_path):
    print("Making Directory: " + testset_path)
    os.makedirs(testset_path)
  if not os.path.exists(trainset_path):
    print("Making Directory: " + trainset_path)
    os.makedirs(trainset_path)

  print("Current Dataset Path: " + originals_path) # where am I
  l = [f for f in os.listdir(originals_path)]
  print(l)
  idx = np.random.choice(len(l), int(test_ratio * len(l)), replace=False)
  print("TestSet Count: " + str(len(idx)) + "\n")
  for i in range(len(l)):
    # copying files to generate training/testing set
    # May takes longer time
    if i in idx: # For chosen test set
      shutil.copyfile(os.path.join(originals_path, l[i]),
                      os.path.join(testset_path, l[i]))
    else:
      shutil.copyfile(os.path.join(originals_path, l[i]),
                      os.path.join(trainset_path, l[i]))

In [ ]:
# from matplotlib import pyplot as plt
# midi_name = os.path.splitext(os.path.basename('/content/drive/My Drive/453/dataset/pop/pop_midi/Backstreet_Boys_-_As_Long_As_You_Love_Me.mid'))[0]
# multitrack = Multitrack(beat_resolution=24, name=midi_name)
# pm = pretty_midi.PrettyMIDI('/content/drive/My Drive/453/dataset/pop/pop_midi/Backstreet_Boys_-_As_Long_As_You_Love_Me.mid')
# multitrack.parse_pretty_midi(pm)
# # Parse a MIDI file to a `pypianoroll.Multitrack` instance
# fig, axs = multitrack.plot()
# fig.set_size_inches(30, 40)
# plt.show()
# for track in multitrack.tracks:
#   print(track.get_active_pitch_range())
# midi_name = os.path.splitext(os.path.basename('/content/drive/My Drive/453/dataset/pop/pop_midi/Backstreet_Boys_-_As_Long_As_You_Love_Me.mid'))[0]
# multitrack = Multitrack(beat_resolution=24, name = midi_name)
# multitrack.parse_midi('/content/drive/My Drive/453/dataset/pop/pop_midi/Backstreet_Boys_-_As_Long_As_You_Love_Me.mid')
# print(multitrack)
# multitrack.write('/test3')


# category_list = {'Piano': [], 'Drums': []}
# program_dict = {'Piano': 0, 'Drums': 0}

# for idx, track in enumerate(multitrack.tracks):
#     if track.is_drum:
#         category_list['Drums'].append(idx)
#     else:
#         category_list['Piano'].append(idx)
# print('\n')
# print(category_list['Drums'])
# print(category_list['Piano'])

# test = multitrack[category_list['Piano']].get_merged_pianoroll()
# print(test)

tracks : [AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe, AsLongAsYouLoveMe],
tempo : [98.99993565 98.99993565 98.99993565 ... 98.99993565 98.99993565
 98.99993565],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : Backstreet_Boys_-_As_Long_As_You_Love_Me


[9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


AttributeError: ignored

In [ ]:
"""2. convert_clean.py"""

def make_sure_path_exists(path):
    """Create all intermediate-level directories if the given path does not
    exist"""
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise


def get_midi_path(root):
    """Return a list of paths to MIDI files in `root` (recursively)"""
    filepaths = []
    for dirpath, _, filenames in os.walk(root):
        for filename in filenames:
            if filename.endswith('.mid'):
                filepaths.append(os.path.join(dirpath, filename))
    return filepaths


def get_midi_info(pm):
    """Return useful information from a pretty_midi.PrettyMIDI instance"""
    if pm.time_signature_changes:
        pm.time_signature_changes.sort(key=lambda x: x.time)
        first_beat_time = pm.time_signature_changes[0].time
    else:
        first_beat_time = pm.estimate_beat_start()

    tc_times, tempi = pm.get_tempo_changes()

    if len(pm.time_signature_changes) == 1:
        time_sign = '{}/{}'.format(pm.time_signature_changes[0].numerator,
                                   pm.time_signature_changes[0].denominator)
    else:
        time_sign = None

    midi_info = {
        'first_beat_time': first_beat_time,
        'num_time_signature_change': len(pm.time_signature_changes),
        'time_signature': time_sign,
        'tempo': tempi[0] if len(tc_times) == 1 else None
    }

    return midi_info

## Time signiture should not effect when generating the song
## This is part of song
# def midi_filter(midi_info):
#     """Return True for qualified midi files and False for unwanted ones"""
#     elif midi_info['num_time_signature_change'] > 1:
#         return False
#     elif midi_info['time_signature'] not in ['4/4']:
#         return False
#     return True


def get_merged(multitrack):
    """Return a `pypianoroll.Multitrack` instance with piano-rolls merged to
    five tracks (Bass, Drums, Guitar, Piano and Strings)"""
    category_list = {'Bass': [], 'Drums': [], 'Guitar': [], 'Piano': [], 'Strings': []}
    program_dict = {'Piano': 0, 'Drums': 0, 'Guitar': 24, 'Bass': 32, 'Strings': 48}

    for idx, track in enumerate(multitrack.tracks):
        if track.is_drum:
            category_list['Drums'].append(idx)
        elif track.program // 8 == 0:
            category_list['Piano'].append(idx)
        elif track.program // 8 == 3:
            category_list['Guitar'].append(idx)
        elif track.program // 8 == 4:
            category_list['Bass'].append(idx)
        else:
            category_list['Strings'].append(idx)

    tracks = []
    for key in category_list:
        if category_list[key]:
            merged = multitrack[category_list[key]].get_merged_pianoroll()
            tracks.append(Track(merged, program_dict[key], key == 'Drums', key))
        else:
            tracks.append(Track(None, program_dict[key], key == 'Drums', key))
   return Multitrack(None, tracks, multitrack.tempo, multitrack.downbeat, multitrack.beat_resolution, multitrack.name)


def converter(filepath, converter_path):
    """Save a multi-track piano-roll converted from a MIDI file to target
    dataset directory and update MIDI information to `midi_dict`"""

    try:
      midi_name = os.path.splitext(os.path.basename(filepath))[0]
      multitrack = Multitrack(beat_resolution=24, name=midi_name)

      pm = pretty_midi.PrettyMIDI(filepath)
      midi_info = get_midi_info(pm)
      multitrack.parse_pretty_midi(pm)
      multitrack.trim_trailing_silence() # remove trailing silence
      merged = get_merged(multitrack)
      # merged = multitrack

      make_sure_path_exists(converter_path)
      merged.save(os.path.join(converter_path, midi_name + '.npz'))

      return [midi_name, midi_info]

    except:
        return None


def main():
    """Main function of the converter"""
    for i in range(2): # have training and testing set for each dataset
      for dataset in DATASET_NAMES: # for each dataset
        dataset_path = os.path.join(ROOT_PATH, dataset, TEMP_SPLITSET_NAMES[i])
        print("Converting: " + dataset_path)

        # set paths
        converter_path = os.path.join(dataset_path, 'converter')
        cleaner_path = os.path.join(dataset_path, 'cleaner')
        midi_paths = get_midi_path(os.path.join(dataset_path, 'origin_midi'))

        # create converter and cleaner path if needed
        if not os.path.exists(converter_path):
          print("Making Directory: " + converter_path)
          os.makedirs(converter_path)
        if not os.path.exists(cleaner_path):
          print("Making Directory: " + cleaner_path)
          os.makedirs(cleaner_path)

        # Get key-value pairs of midi files
        midi_dict = {}
        kv_pairs = [converter(midi_path, converter_path) for midi_path in midi_paths]
        for kv_pair in kv_pairs:
            if kv_pair is not None:
                midi_dict[kv_pair[0]] = kv_pair[1]

        # Logging
        with open(os.path.join(dataset_path, 'midis.json'), 'w') as outfile:
            json.dump(midi_dict, outfile)

        print("[Done] {} files out of {} have been successfully converted".format(len(midi_dict), len(midi_paths)))

        with open(os.path.join(dataset_path, 'midis.json')) as infile:
            midi_dict = json.load(infile)
        count = 0
        make_sure_path_exists(cleaner_path)
        midi_dict_clean = {}
        for key in midi_dict:
            # if midi_filter(midi_dict[key]):
                midi_dict_clean[key] = midi_dict[key]
                count += 1
                shutil.copyfile(os.path.join(converter_path, key + '.npz'),
                                os.path.join(cleaner_path, key + '.npz'))

        with open(os.path.join(dataset_path, 'midis_clean.json'), 'w') as outfile:
            json.dump(midi_dict_clean, outfile)

        print("[Done] {} files out of {} have been successfully cleaned\n".format(count, len(midi_dict)))


if __name__ == "__main__":
    main()


Converting: /content/drive/My Drive/453/dataset/classical/test_temp
Making Directory: /content/drive/My Drive/453/dataset/classical/test_temp/converter
Making Directory: /content/drive/My Drive/453/dataset/classical/test_temp/cleaner
[Done] 22 files out of 22 have been successfully converted
[Done] 22 files out of 22 have been successfully cleaned

Converting: /content/drive/My Drive/453/dataset/jazz/test_temp
Making Directory: /content/drive/My Drive/453/dataset/jazz/test_temp/converter
Making Directory: /content/drive/My Drive/453/dataset/jazz/test_temp/cleaner
[Done] 55 files out of 55 have been successfully converted
[Done] 55 files out of 55 have been successfully cleaned

Converting: /content/drive/My Drive/453/dataset/pop/test_temp
Making Directory: /content/drive/My Drive/453/dataset/pop/test_temp/converter
Making Directory: /content/drive/My Drive/453/dataset/pop/test_temp/cleaner
[Done] 30 files out of 30 have been successfully converted
[Done] 30 files out of 30 have been su

In [ ]:
"""3. choose the clean midi from original sets"""

for i in range(2): # have training and testing set for each dataset
  for dataset in DATASET_NAMES: # for each dataset
    dataset_path = os.path.join(ROOT_PATH, dataset, TEMP_SPLITSET_NAMES[i])
    cleaner_midi_path = os.path.join(dataset_path, 'cleaner_midi')
    print("Choose Clean MIDI of: " + dataset_path)

    if not os.path.exists(cleaner_midi_path):
        os.makedirs(cleaner_midi_path)
        print("Making Directory: " + cleaner_midi_path)
    
    l = [f for f in os.listdir(os.path.join(dataset_path, 'cleaner'))]
    print(l)
    print(str(len(l)) + '\n')
    for index in l:
        shutil.copy(os.path.join(dataset_path, 'origin_midi', os.path.splitext(index)[0] + '.mid'),
                    os.path.join(cleaner_midi_path, os.path.splitext(index)[0] + '.mid'))

Choose Clean MIDI of: /content/drive/My Drive/453/dataset/classical/test_temp
Making Directory: /content/drive/My Drive/453/dataset/classical/test_temp/cleaner_midi
['bach-bourree-in-e-minor-guitar.npz', 'Boccherini-Minuet.npz', 'Boccherini-Minuet-Oboe-and-Piano.npz', 'Boccherini-Minuet-Soprano-Sax-and-Piano.npz', 'Boccherini-Minuet-Tenor-Sax-and-Piano.npz', 'brahms-lullaby-wiegenlied.npz', 'brahms-st-anthony-chorale-theme-trio.npz', 'sellengers-round-virginal.npz', 'Golliwogs-Cakewalk.npz', 'edvard-grieg-peer-gynt1-morning-mood.npz', 'Songwithoutwords06.npz', 'Songwithoutwords34.npz', 'mozart-symphony40-1.npz', 'mozart-piano-concerto-21-2-elvira-madigan-piano-solo.npz', 'Eine-Kleine-Nachtmusik1-Clarinet-A.npz', 'saint-saens-carnival-of-the-animals-the-swan-viola-piano.npz', 'Satie-Gymnopedie1-bassoon-piano.npz', 'about-strange-lands-and-people.npz', 'Third-Mode-Melody.npz', 'Tchaikovsky-Waltz-op39-no8.npz', 'wagner-bridal-chorus.npz', 'romance-anonimo.npz']
22

Choose Clean MIDI of: /

In [ ]:
"""4. merge and crop"""

for set_i in range(2): # have training and testing set for each dataset
  for dataset in DATASET_NAMES: # for each dataset
    dataset_path = os.path.join(ROOT_PATH, dataset, TEMP_SPLITSET_NAMES[set_i])
    print("Merge and Crop: " + dataset_path)

    if not os.path.exists(os.path.join(dataset_path, 'cleaner_midi_gen')):
        os.makedirs(os.path.join(dataset_path, 'cleaner_midi_gen'))
        print("Making Directory: " + os.path.join(dataset_path, 'cleaner_midi_gen'))
    if not os.path.exists(os.path.join(dataset_path, 'cleaner_npy')):
        os.makedirs(os.path.join(dataset_path, 'cleaner_npy'))
        print("Making Directory: " + os.path.join(dataset_path, 'cleaner_npy'))

    l = [f for f in os.listdir(os.path.join(dataset_path, 'cleaner_midi'))]
    print(l)
    count = 0
    for i in range(len(l)):
        try:
            multitrack = Multitrack(beat_resolution=4, name=os.path.splitext(l[i])[0])
            # multitrack = Multitrack(beat_resolution=24, name=os.path.splitext(l[i])[0])
            x = pretty_midi.PrettyMIDI(os.path.join(dataset_path, 'cleaner_midi', l[i]))
            multitrack.parse_pretty_midi(x)

            category_list = {'Piano': [], 'Drums': []}
            program_dict = {'Piano': 0, 'Drums': 0}

            for idx, track in enumerate(multitrack.tracks): # removing drum
                if track.is_drum:
                    category_list['Drums'].append(idx)
                else:
                    category_list['Piano'].append(idx)
            tracks = []
            merged = multitrack[category_list['Piano']].get_merged_pianoroll()
            print(merged.shape)

            pr = get_bar_piano_roll(merged)
            print(pr.shape)
            pr_clip = pr[:, :, 24:108] ## Default
            print(pr_clip.shape)
            if int(pr_clip.shape[0] % 4) != 0:
                pr_clip = np.delete(pr_clip, np.s_[-int(pr_clip.shape[0] % 4):], axis=0)
            pr_re = pr_clip.reshape(-1, 64, 84, 1)
            print(pr_re.shape)
            save_midis(pr_re, os.path.join(dataset_path, 'cleaner_midi_gen', os.path.splitext(l[i])[0] +
                                          '.mid'))
            np.save(os.path.join(dataset_path, 'cleaner_npy', os.path.splitext(l[i])[0] + '.npy'), pr_re)
        except:
            count += 1
            print('Wrong', l[i])
            continue
    print("Error Count: " + str(count) + '\n')

Streaming output truncated to the last 5000 lines.
(20, 64, 84)
(20, 64, 84, 1)
(2688, 128)
(42, 64, 128)
(42, 64, 84)
(40, 64, 84, 1)
(1872, 128)
(29, 64, 128)
(29, 64, 84)
(28, 64, 84, 1)
(1632, 128)
(25, 64, 128)
(25, 64, 84)
(24, 64, 84, 1)
(1532, 128)
(23, 64, 128)
(23, 64, 84)
(20, 64, 84, 1)
(1620, 128)
(25, 64, 128)
(25, 64, 84)
(24, 64, 84, 1)
(1280, 128)
(20, 64, 128)
(20, 64, 84)
(20, 64, 84, 1)
(2052, 128)
(32, 64, 128)
(32, 64, 84)
(32, 64, 84, 1)
(1028, 128)
(16, 64, 128)
(16, 64, 84)
(16, 64, 84, 1)
(1216, 128)
(19, 64, 128)
(19, 64, 84)
(16, 64, 84, 1)
(1240, 128)
(19, 64, 128)
(19, 64, 84)
(16, 64, 84, 1)
(4368, 128)
(68, 64, 128)
(68, 64, 84)
(68, 64, 84, 1)
(1776, 128)
(27, 64, 128)
(27, 64, 84)
(24, 64, 84, 1)
(704, 128)
(11, 64, 128)
(11, 64, 84)
(8, 64, 84, 1)
(1724, 128)
(26, 64, 128)
(26, 64, 84)
(24, 64, 84, 1)
(972, 128)
(15, 64, 128)
(15, 64, 84)
(12, 64, 84, 1)
(1424, 128)
(22, 64, 128)
(22, 64, 84)
(20, 64, 84, 1)
Error Count: 0

Merge and Crop: /content/dr

In [ ]:
"""5. concatenate into a big binary numpy array file"""

for set_i in range(2): # have training and testing set for each dataset
  for dataset in DATASET_NAMES: # for each dataset
    dataset_path = os.path.join(ROOT_PATH, dataset, TEMP_SPLITSET_NAMES[set_i])
    print("Concat into a big Binary Numpy Array File: " + dataset_path)

    l = [f for f in os.listdir(os.path.join(dataset_path, 'cleaner_npy'))]
    print(l)

    train = np.load(os.path.join(dataset_path, 'cleaner_npy', l[0]))
    print(train.shape, np.max(train))
    for i in range(1, len(l)):
        print(i, l[i])
        t = np.load(os.path.join(dataset_path, 'cleaner_npy', l[i]))
        train = np.concatenate((train, t), axis=0)
    print(train.shape)

    np_name = ""
    if set_i == 0:
      np_name = dataset + "_test_piano.npy"
    elif set_i == 1:
      np_name = dataset + "_train_piano.npy"
    else:
      print("ERROR\n")
      break

    np.save(os.path.join(dataset_path, np_name), (train > 0.0))
    print("\n")

Concat into a big Binary Numpy Array File: /content/drive/My Drive/453/dataset/classical/test_temp
['bach-bourree-in-e-minor-guitar.npy', 'Boccherini-Minuet.npy', 'Boccherini-Minuet-Oboe-and-Piano.npy', 'Boccherini-Minuet-Soprano-Sax-and-Piano.npy', 'Boccherini-Minuet-Tenor-Sax-and-Piano.npy', 'brahms-lullaby-wiegenlied.npy', 'brahms-st-anthony-chorale-theme-trio.npy', 'sellengers-round-virginal.npy', 'Golliwogs-Cakewalk.npy', 'edvard-grieg-peer-gynt1-morning-mood.npy', 'Songwithoutwords06.npy', 'Songwithoutwords34.npy', 'mozart-symphony40-1.npy', 'mozart-piano-concerto-21-2-elvira-madigan-piano-solo.npy', 'Eine-Kleine-Nachtmusik1-Clarinet-A.npy', 'saint-saens-carnival-of-the-animals-the-swan-viola-piano.npy', 'Satie-Gymnopedie1-bassoon-piano.npy', 'about-strange-lands-and-people.npy', 'Third-Mode-Melody.npy', 'Tchaikovsky-Waltz-op39-no8.npy', 'wagner-bridal-chorus.npy', 'romance-anonimo.npy']
(12, 64, 84, 1) 109
1 Boccherini-Minuet.npy
2 Boccherini-Minuet-Oboe-and-Piano.npy
3 Boccheri

In [ ]:
"""6. separate numpy array file into single phrases"""
## The final file
FINAL_NAME = ['test', 'train']

for set_i in range(2): # have training and testing set for each dataset
  for dataset in DATASET_NAMES: # for each dataset
    dataset_path = os.path.join(ROOT_PATH, dataset, TEMP_SPLITSET_NAMES[set_i])
    print("Separate Numpy Array into single Phrases: " + dataset_path)

    final_path = os.path.join(ROOT_PATH, dataset, FINAL_NAME[set_i])
    print("Final Path is: ", final_path)

    combined_file_path = ""
    if set_i == 0:
      combined_file_path = dataset + "_test_piano.npy"
    elif set_i == 1:
      combined_file_path = dataset + "_train_piano.npy"
    else:
      print("ERROR\n")
      break
    
    if not os.path.exists(final_path):
        os.makedirs(final_path)
        print("Making Directory: " + final_path)
    x = np.load(os.path.join(dataset_path, combined_file_path))
    print(x.shape)

    count = 0
    for i in range(x.shape[0]):
        if np.max(x[i]):
            count += 1
            np.save(os.path.join(final_path, (dataset + '_piano_' + FINAL_NAME[set_i] + '_{}.npy'.format(i+1))), x[i])
        if i % 100 == 0:
          print("Finish Converting: " + str(i))
      # if count == 11216:
      #     break
    print("Total Phrases: " + str(count) + "\n")

Separate Numpy Array into single Phrases: /content/drive/My Drive/453/dataset/classical/test_temp
Final Path is:  /content/drive/My Drive/453/dataset/classical/test
Making Directory: /content/drive/My Drive/453/dataset/classical/test
(328, 64, 84, 1)
Finish Converting: 0
Finish Converting: 100
Finish Converting: 200
Finish Converting: 300
Total Phrases: 328

Separate Numpy Array into single Phrases: /content/drive/My Drive/453/dataset/jazz/test_temp
Final Path is:  /content/drive/My Drive/453/dataset/jazz/test
Making Directory: /content/drive/My Drive/453/dataset/jazz/test
(1492, 64, 84, 1)
Finish Converting: 0
Finish Converting: 100
Finish Converting: 200
Finish Converting: 300
Finish Converting: 400
Finish Converting: 500
Finish Converting: 600
Finish Converting: 700
Finish Converting: 800
Finish Converting: 900
Finish Converting: 1000
Finish Converting: 1100
Finish Converting: 1200
Finish Converting: 1300
Finish Converting: 1400
Total Phrases: 1479

Separate Numpy Array into single 

## Training

In [ ]:
!python main.py --dataset_dir='/content/drive/My Drive/453/dataset2' \
                --dataset_A_dir='pop' --dataset_B_dir='classical' \
                --checkpoint_dir='/content/drive/My Drive/453/checkpoint2' \
                --sample_dir='/content/drive/My Drive/453/samples2' \
                --test_dir='/content/drive/My Drive/453/test2' \
                --log_dir='/content/drive/My Drive/453/log2' \
                --type='cyclegan' --model='base' --sigma_d=1 --phase='train' \
                --epoch=200 --epoch_step=100 --lr=0.0001

Streaming output truncated to the last 5000 lines.
Epoch: [13] [ 196/ 213] time: 5472.1783, d_loss:   0.49, G_loss:   0.86
++++++++++G_loss_A2B:   0.62 G_loss_B2A:   0.62 Cycle_loss:   0.38 DA_loss:   0.24 DB_loss:   0.24
Epoch: [13] [ 197/ 213] time: 5472.5999, d_loss:   0.50, G_loss:   0.87
++++++++++G_loss_A2B:   0.58 G_loss_B2A:   0.59 Cycle_loss:   0.30 DA_loss:   0.25 DB_loss:   0.25
Epoch: [13] [ 198/ 213] time: 5473.0154, d_loss:   0.49, G_loss:   0.74
++++++++++G_loss_A2B:   0.51 G_loss_B2A:   0.49 Cycle_loss:   0.26 DA_loss:   0.24 DB_loss:   0.25
Epoch: [13] [ 199/ 213] time: 5473.4213, d_loss:   0.49, G_loss:   0.86
++++++++++G_loss_A2B:   0.55 G_loss_B2A:   0.58 Cycle_loss:   0.27 DA_loss:   0.25 DB_loss:   0.24
Epoch: [13] [ 200/ 213] time: 5473.8261, d_loss:   0.48, G_loss:   0.84
++++++++++G_loss_A2B:   0.60 G_loss_B2A:   0.60 Cycle_loss:   0.36 DA_loss:   0.24 DB_loss:   0.25
Epoch: [13] [ 201/ 213] time: 5474.2404, d_loss:   0.49, G_loss:   0.86
++++++++++G_loss_A2B: 

## Testing

In [ ]:
!python main.py --dataset_dir='/content/drive/My Drive/453/dataset2' \
                --dataset_A_dir='pop' --dataset_B_dir='classical' \
                --checkpoint_dir='/content/drive/My Drive/453/checkpoint2' \
                --sample_dir='/content/drive/My Drive/453/samples2' \
                --test_dir='/content/drive/My Drive/453/test2' \
                --log_dir='/content/drive/My Drive/453/log2' \
                --type='cyclegan' --model='base' --sigma_d=1 --phase='test' \
                --which_direction="BtoA"

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2020-04-21 13:14:55.073153: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-04-21 13:14:55.078107: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000189999 Hz
2020-04-21 13:14:55.078306: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2bc8f40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-21 13:14:55.078333: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-04-21 13:14:55.080163: I t

In [ ]:
drive.flush_and_unmount()